# DSE 203 - Assignment 3
## Group 5 - Christopher Vanhook, Vaaruni Desai, Zufeshan Imran
### <font color='red'>Query - List all the unincorporated places where there is a cafe</font>

### Install the required libraries for this assignment

In [1]:
!pip install fuzzywuzzy python-levenshtein psycopg2-binary jsonpath_ng

### Import the required libraries

In [2]:
import json
import time
import psycopg2
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from jsonpath_ng.ext import parse

### We're looking at two files. A JSON file containing information about places in San Diego and a CSV file containing information about businesses in and around San Diego

In [3]:
json_file = 'statisticalAtlas.json'
csv_file = 'nourish_public_ca_business.csv'

### Import CSV into pgadmin and query the required results


#### Connect to pgadmin host - The cell takes user input

In [4]:
database = input("Enter database name:")
host = input("\nEnter host name:")
user = input("\nEnter user name:")
password = input("\nEnter password:")
port = input("\nEnter port number:")

conn = psycopg2.connect(database=database,
                        host=host,
                        user=user,
                        password=password,
                        port=port)

cur = conn.cursor()

Enter database name: postgres

Enter host name: localhost

Enter user name: postgres

Enter password: password

Enter port number: 5432


#### Import csv data into pgadmin. 
1. Drop the table if it already exists
2. Create a table 
3. Copy/import csv data into respective columns in the created table 

In [5]:
# Drop the table if it already exists
DROP_TABLE = """DROP table if EXISTS nourish_public """
cur.execute(DROP_TABLE)
conn.commit()

In [6]:
# Create the table
CREATE_TABLE = """CREATE table IF NOT EXISTS nourish_public
(id bigserial primary key,
 name varchar,
 address varchar,
 avg_rating varchar,
 zip varchar(5),
 categories text[],
 city varchar
);"""
cur.execute(CREATE_TABLE)
conn.commit()

In [7]:
# Import data from csv file into postgres database into the created table
IMPORT_CSV = """COPY nourish_public(name,address,avg_rating,zip,categories,city) FROM '/tmp/nourish_public_ca_business.csv' DELIMITER ',' CSV HEADER;"""
cur.execute(IMPORT_CSV)
conn.commit()
cur.close()

#### SQL Query to extract places where there is a cafe

In [8]:
# Open the cursor, execute the SQL query and close the cursor
start_time = time.time()
cur =conn.cursor()
SQL_QUERY = """SELECT * FROM (SELECT *, unnest(categories) as cat FROM nourish_public) s WHERE lower(cat) LIKE '%cafe%';"""
cur.execute(SQL_QUERY)
records = cur.fetchall()
col_names = [desc[0] for desc in cur.description]
cur.close()
print("SQL Query Runetime: %s microseconds" % ((time.time() - start_time)*100000))

SQL Query Runetime: 402.6651382446289 microseconds


### Read JSON, query the required results

#### Load in JSON file using json library

In [9]:
with open(json_file, "r") as f:
    json_data = json.load(f)

#### JSON Query to extract Unincorporated Places

In [10]:
# JSONPath expression to get all the Unincorporated places
start_time = time.time()
jsonpath_expr = parse("$.['Unincorporated Places']")
matches = [match.value for match in jsonpath_expr.find(json_data)]
print("JSON Query Runetime: %s microseconds" % ((time.time() - start_time)*100000))

JSON Query Runetime: 7798.194885253906 microseconds


### Function to create Hash tables for both SQL and JSON data and JOIN them by matching the HASH table keys

Regarding the json_csv_hash_join function -
Here, the sql key is used to check if it exists in json hash table. Then the json data for that particular key is extracted and stored in result variable. There are cases where the names of cities don't match exactly. 
For example - JSON data has listed Mount Laguna as 'Mount Laguna' whereas SQL data (csv file) has it listed as Mt. Laguna. 
For such cases, fuzzywuzzy comes in handy. With a threshold of 85, we were able to extract all the matching records we wanted. 

Threshold can be adjusted based on the differences in the files that are dealt with.

In [11]:
# Function to create JSON hash table given the json_data and the key name
def json_hash(json_file, json_parser_query, json_column_retrieve):
    jsonpath_expr = parse(f"$.['{json_parser_query}']")
    matches = [match.value for match in jsonpath_expr.find(json_file)]
    start_time = time.time()
    json_hash = {}

    for entry in matches[0]:
        city_name = entry.get(json_column_retrieve)
        if city_name is not None:
            if city_name not in json_hash:
                json_hash[city_name] = []
            json_hash[city_name].append(entry)
    print("JSON Hash table creation runtime: %s microseconds" % ((time.time() - start_time)*100000))

    return json_hash

# Function to create SQL hash table given the sql_data and the index of the item to be used as key
def sql_hash(csv_file, column_name):
    start_time = time.time()
    sql_hash_dict = {}
    for entry in records:
        city_name = entry[col_names.index(column_name)]
        if city_name is not None:
            if city_name not in sql_hash_dict:
                sql_hash_dict[city_name] = []
            sql_hash_dict[city_name].append(entry)
    print("SQL Hash table creation runtime: %s microseconds" % ((time.time() - start_time)*100000))
    return sql_hash_dict

# Join data based on the matching keys and measure the processing time
def json_csv_hash_join(json_file, sql_file, column_csv, column_json, json_parser, keys_to_return = {'City Name', 'State', 'County'}):
    sql_hash_column = sql_hash(json_file, column_csv)
    json_hash_column = json_hash(json_file, json_parser, column_json)
    start_time = time.time()
    threshold = 85
    matching_keys = []
    keys_to_return = keys_to_return

    for sql_key in sql_hash_column:
        match = process.extractOne(sql_key, json_hash_column.keys(), scorer=fuzz.token_sort_ratio)
        if match[1] >= threshold:
            matching_keys.append({key: json_hash_column[match[0]][0][key] for key in json_hash_column[match[0]][0].keys() & keys_to_return})
    time_taken = (time.time() - start_time)
    print(f"Joining Hash tables runtime: {time_taken * 100000} microseconds")
    return matching_keys

In [19]:
result = json_csv_hash_join(json_file = json_data, sql_file = records, column_csv = 'city', column_json = 'City Name', json_parser = 'Unincorporated Places')

print(f"\nList of Unincorporated areas where there is a cafe \n{json.dumps(result, sort_keys=True, indent=4)}")

SQL Hash table creation runtime: 5.8650970458984375 microseconds
JSON Hash table creation runtime: 2.7418136596679688 microseconds
Joining Hash tables runtime: 381.32667541503906 microseconds

List of Unincorporated areas where there is a cafe 
[
    {
        "City Name": "Rancho Santa Fe",
        "County": "San Diego",
        "State": "California"
    },
    {
        "City Name": "Pala",
        "County": "San Diego",
        "State": "California"
    },
    {
        "City Name": "Mount Laguna",
        "County": "San Diego",
        "State": "California"
    }
]
